In [ ]:
# to do , I am using 2 different computational graphs, so need 2 different sessions


In [3]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
import six
#import tensorflow as tf

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

In [4]:
num_classes=4
data_base_directory='data/datav2/'
alphabet_dict={0:'A' , 1:'B' , 2:'C' , 3:'D' , 4:'E', 5:'F' , 6:'G' ,7 :'H' , 8:'I', 9:'J'}


In [5]:
def process_function(input):
  
  

  input=data_base_directory + str(input ,'utf-8')
  #print(str(input,'utf-8'))

 

  for i in range(num_classes):
     if alphabet_dict[i] in input:
         y=int(i)

  #if 'A' in str(input,'utf-8'):
   #     y=0
 


  
    
  return  input,y


In [6]:
logfilepath='data/dataset.txt'
dataset = tf.data.TextLineDataset(logfilepath).shuffle(buffer_size=200) #.map(map_function)

dataset = dataset.map(
    lambda filename: tuple(tf.py_func(
        process_function, [filename],  [tf.string,tf.int64])))

dataset=dataset.batch(1)


# In[25]:


iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()


# In[28]:

with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(200):
      value = sess.run(next_element[0])
      print(value)


[b'data/datav2/20_48_00026A.wav']
[b'data/datav2/20_48_00010A.wav']
[b'data/datav2/20_50_46072A.wav']
[b'data/datav2/20_50_46134A.wav']
[b'data/datav2/20_54_21016B.wav']
[b'data/datav2/20_50_46019A.wav']
[b'data/datav2/20_48_00028A.wav']
[b'data/datav2/20_50_46000A.wav']
[b'data/datav2/20_50_46025A.wav']
[b'data/datav2/20_50_46013A.wav']
[b'data/datav2/20_50_46086A.wav']
[b'data/datav2/20_54_21000B.wav']
[b'data/datav2/20_54_21023B.wav']
[b'data/datav2/20_50_46096A.wav']
[b'data/datav2/20_50_46037A.wav']
[b'data/datav2/20_54_21041B.wav']
[b'data/datav2/20_50_46065A.wav']
[b'data/datav2/20_50_46069A.wav']
[b'data/datav2/20_50_46116A.wav']
[b'data/datav2/20_50_46055A.wav']
[b'data/datav2/20_56_39001B.wav']
[b'data/datav2/20_50_46053A.wav']
[b'data/datav2/20_50_46127A.wav']
[b'data/datav2/20_50_46007A.wav']
[b'data/datav2/20_50_46051A.wav']
[b'data/datav2/20_54_21011B.wav']
[b'data/datav2/20_54_21029B.wav']
[b'data/datav2/20_50_46042A.wav']
[b'data/datav2/20_50_46011A.wav']
[b'data/datav2

In [7]:
# Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 2
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 128 # MNIST data input (img shape: 28*28)
#num_classes = 5 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.int32, [None])


In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 =  tf.nn.elu (tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 256 neurons
    layer_2 =  tf.nn.elu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [10]:
# Construct model
Logits = neural_net(X)

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
 #   logits=logits, labels=Y))
prediction = tf.nn.softmax(Logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=Logits))
train_op = optimizer.minimize(cost)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(Logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [12]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(num_steps):
        avg_cost = 0.
        
        
        sess.run(iterator.initializer)
        while True:
            try:
                Next_element=sess.run(next_element)
                input_wav_file_name=str(Next_element[0][0],'utf-8')
                print(input_wav_file_name)
                
                examples_batch = vggish_input.wavfile_to_examples(input_wav_file_name)
                print(examples_batch)

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor('vggish_pca_params.npz')
                
                vggish_slim.define_vggish_slim(training=False)
                vggish_slim.load_vggish_slim_checkpoint(sess, 'vggish_model.ckpt')
                features_tensor = sess.graph.get_tensor_by_name(
                vggish_params.INPUT_TENSOR_NAME)
                embedding_tensor = sess.graph.get_tensor_by_name(
                vggish_params.OUTPUT_TENSOR_NAME)

    # Run inference and postprocessing.
                [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
                print(embedding_batch)
                postprocessed_batch = pproc.postprocess(embedding_batch)
                print(postprocessed_batch)
    
    

              # train single example
                _ , c = sess.run([optimizer, cost], feed_dict={x: postprocessed_batch,y: Next_element[1] , seq_len:Next_element[2]})
            
                 
                print(c)
                #print(pre)
      #          print(p)
               
               
              
            except tf.errors.OutOfRangeError:
              break
   
        
                
    # Save model weights to disk
   # save_path = saver.save(sess, model_path)
    #print("Model saved in file: %s" % save_path)   

data/datav2/20_50_46038A.wav
[[[-1.12978737 -0.484816   -0.50931633 ... -2.55864364 -2.56120469
   -2.73118087]
  [-0.32528157 -1.18447556 -0.18761478 ... -3.54096869 -3.47272872
   -3.62046983]
  [-1.12022411 -0.55269468 -0.20893125 ... -3.7981272  -3.71390192
   -3.70605061]
  ...
  [-1.10347048 -1.24851905  0.68397672 ... -2.99936885 -3.06113702
   -3.96019784]
  [-1.41108985 -0.98967099  0.66131641 ... -3.38197333 -3.37824862
   -3.51605169]
  [-2.14763618 -1.58263743  0.7749115  ... -3.68581035 -3.56579007
   -3.66093811]]

 [[-1.93383517 -1.5147824   0.64292248 ... -3.79676084 -3.66523049
   -3.8893739 ]
  [-1.55679351 -1.58949676  0.81459531 ... -3.83428883 -3.42111123
   -3.60055281]
  [-0.89059472 -2.02527303  0.95151461 ... -3.93778216 -3.69208625
   -3.74298804]
  ...
  [-0.96371902 -1.12399775  0.18110006 ... -3.3748769  -2.9571567
   -2.973651  ]
  [-1.99817251 -0.66230004  0.23193735 ... -3.39231654 -3.06285541
   -2.90667629]
  [-1.36999589 -0.86164656  0.18305598 ... -3

NameError: name 'x' is not defined

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))